In [1]:
import os 

In [2]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file: Path
    unzip_dir: Path



In [6]:
from src.chicken_disease_classification.constants import *
from src.chicken_disease_classification.utils.common import read_yaml, create_directories

In [7]:
class configManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
    

In [8]:
import os 
import zipfile
import urllib.request as request
from src.chicken_disease_classification.utils.common import get_size
from src.chicken_disease_classification import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(url=self.config.source_URL, filename=self.config.local_data_file)
                logger.info(f"{filename} downloaded! with the following info: {headers}")
            except Exception as e:
                logger.error(f"Error occurred while downloading the file: {e}")
                raise e  # Raise the exception to stop execution
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        
        # Verify the file exists and has content
        if not os.path.exists(self.config.local_data_file) or os.path.getsize(self.config.local_data_file) == 0:
            raise FileNotFoundError(f"Failed to download file to {self.config.local_data_file}")
            
    def extract_zip_file(self):
        # Check if file exists before trying to extract
        if not os.path.exists(self.config.local_data_file):
            raise FileNotFoundError(f"Zip file not found at {self.config.local_data_file}")
            
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"File extracted successfully to {unzip_path}")
        except zipfile.BadZipFile:
            logger.error(f"File is not a valid zip file: {self.config.local_data_file}")
            raise

In [10]:
config = configManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()
try:
    print("Data ingestion completed successfully")  
except Exception as e:
    logger.error(f"An error occurred during data ingestion completion message: {e}")

[2025-03-20 16:47:43,738: INFO: common: Reading YAML file from: config/config.yaml]
[2025-03-20 16:47:43,739: INFO: common: YAML file read successfully.]
[2025-03-20 16:47:43,739: INFO: common: Reading YAML file from: params.yaml]
[2025-03-20 16:47:43,739: INFO: common: YAML file read successfully.]
[2025-03-20 16:47:43,740: INFO: common: created directory at: artifacts]
[2025-03-20 16:47:43,740: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-20 16:47:47,207: INFO: 3464350194: artifacts/data_ingestion/data.zip downloaded! with the following info: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 378F:259B43:D4650:1